In [1]:
from cpd_search import *  # using cpd/original/cpd_search.py
from tqdm import tqdm
import math

In [2]:
def perm_parity(P):
    n=len(P)
    assert set(P)==set(range(n))
    seen=[False]*n
    out=0
    for i in range(n):
        if not seen[i]:
            cyc_len=0
            h=i
            while not seen[h]:
                seen[h]=True
                h=P[h]
                cyc_len+=1
            out^=(cyc_len-1)%2
    return out

In [3]:
def wedge_prod(vecs,MOD):
    assert isinstance(vecs,tuple)
    D=len(vecs)
    assert D>0
    n,=vecs[0].shape
    assert all(v.shape==(n,) for v in vecs)
    out=0
    for perm in it.permutations(range(D)):
        out=(out + (-1)**perm_parity(perm) * cpd_eval([tuple(vecs[p] for p in perm)],MOD))%MOD
    return out

In [4]:
wedge_prod(tuple(np.eye(3)),MOD=97)

array([[[ 0.,  0.,  0.],
        [ 0.,  0.,  1.],
        [ 0., 96.,  0.]],

       [[ 0.,  0., 96.],
        [ 0.,  0.,  0.],
        [ 1.,  0.,  0.]],

       [[ 0.,  1.,  0.],
        [96.,  0.,  0.],
        [ 0.,  0.,  0.]]])

In [5]:
def all_tensors(shape,MOD):
    return (
        np.array(v,dtype=int).reshape(shape)
        for v in it.product(range(MOD),repeat=np.prod(shape))
    )

def all_nonzero_tensors(shape,MOD):
    return (
        T
        for T in all_tensors(shape,MOD)
        if (T!=0).any()
    )

In [40]:
def k_th_rref_prune0(T,R,MOD,k,verbose=False):
    n0=T.shape[0]
    if R<n0:
        return NO_CPD_EXISTS
    good_vec_subsets=[]
    for vecs in it.combinations(all_nonzero_tensors((n0,),MOD),k):
        M=np.array(vecs)
        M_T=axis_op0(M,T,MOD)
        if cpd_search(M_T,R-n0+k,MOD,verbose=False) is not None:
            good_vec_subsets.append(vecs)
    
    if verbose:
        print(good_vec_subsets)
        
        # generate minimal list of vector subsets
        # such that their wedge products span a maximal-dimension space;
        # equivalently, the set of linear subspaces spanned by the subsets
        # span all possible k-planes of (F_MOD)**n_0
        cur_mat=np.zeros((0,n0**k),dtype=int)
        subset_basis=[]
        for vecs in good_vec_subsets:
            new_mat=np.concatenate([
                cur_mat,
                wedge_prod(vecs,MOD).reshape((1,-1))
            ],axis=0)
            if mat_rank(new_mat,MOD)>cur_mat.shape[0]:
                cur_mat=new_mat
                subset_basis.append(vecs)
        print(subset_basis)
    
    wedges_mat=np.array([
        wedge_prod(vecs,MOD) for vecs in good_vec_subsets
    ],dtype=int).reshape((len(good_vec_subsets),n0**k))
    if mat_rank(wedges_mat,MOD)>=math.comb(n0,k):
        return CANNOT_DETERMINE_IF_CPD_EXISTS
    return NO_CPD_EXISTS

In [43]:
T=wedge_prod(tuple(np.eye(3,dtype=int)),MOD=2)
print(k_th_rref_prune0(T,R=5,MOD=2,k=2,verbose=True))
del T

[(array([0, 0, 1]), array([0, 1, 0])), (array([0, 0, 1]), array([0, 1, 1])), (array([0, 0, 1]), array([1, 0, 0])), (array([0, 0, 1]), array([1, 0, 1])), (array([0, 0, 1]), array([1, 1, 0])), (array([0, 0, 1]), array([1, 1, 1])), (array([0, 1, 0]), array([0, 1, 1])), (array([0, 1, 0]), array([1, 0, 0])), (array([0, 1, 0]), array([1, 0, 1])), (array([0, 1, 0]), array([1, 1, 0])), (array([0, 1, 0]), array([1, 1, 1])), (array([0, 1, 1]), array([1, 0, 0])), (array([0, 1, 1]), array([1, 0, 1])), (array([0, 1, 1]), array([1, 1, 0])), (array([0, 1, 1]), array([1, 1, 1])), (array([1, 0, 0]), array([1, 0, 1])), (array([1, 0, 0]), array([1, 1, 0])), (array([1, 0, 0]), array([1, 1, 1])), (array([1, 0, 1]), array([1, 1, 0])), (array([1, 0, 1]), array([1, 1, 1])), (array([1, 1, 0]), array([1, 1, 1]))]
[(array([0, 0, 1]), array([0, 1, 0])), (array([0, 0, 1]), array([1, 0, 0])), (array([0, 1, 0]), array([1, 0, 0]))]
None


In [8]:
# copied code from "maximum rank/generating canonical tensors.ipynb"
def all_invle_mats(n,MOD):
    for v in it.product(range(MOD),repeat=n*n):
        M=np.array(v,dtype=int).reshape((n,n))
        if mat_rank(M,MOD)==n:
            yield M

def tensor2code(T):
    '''
    convert tensor T into a hashable object
    '''
    assert isinstance(T,np.ndarray), type(T)
    return (T.shape,tuple(T.flatten()))

def linsys(A,B,MOD,iterate=True):
    '''
    if iter:
        return iterable of all solutions M to A@M=B mod MOD
    else:
        return X,Y such that the set {X+Y@Z : Z}
        parameterizes all such solutions
    '''
    assert len(A.shape)==2, A.shape
    assert len(B.shape)==2, B.shape
    assert A.shape[0]==B.shape[0], (A.shape,B.shape)
    rrefA,P,_,r=mat_row_reduce(A,MOD)
    _,QT,_,_=mat_row_reduce(rrefA.T,MOD)
    Q=QT.T
    reduced_expected=np.zeros(A.shape,dtype=int)
    reduced_expected[range(r),range(r)]=1
    assert np.array_equal((P@A@Q)%MOD,reduced_expected), (P,A,Q,r,(P@A@Q)%MOD)
    # (P@A@Q)@(inv(Q)@M)=P@B
    PB=(P@B)%MOD
    if not (PB[r:,:]==0).all():
        # no solutions
        if iterate:
            return []
        else:
            return None
    # M = Q@cat([PB[:r,:],arbitrary],axis=0)
    #   = Q[:,:r]@PB[:r,:] + Q[:,r:]@arbitrary
    M0,M1=(Q[:,:r]@PB[:r,:])%MOD, Q[:,r:]
    if iterate:
        return (
            (M0+M1@X)%MOD
            for X in all_tensors((M1.shape[1],M0.shape[1]),MOD)
        )
    else:
        return M0,M1

def dfs_canonicals_dim3(
    n_slices,nrows,nclms,MOD,row_ops,
    cur_slices,sol_list
):
    '''
    row_ops: list of allowed row operations on slices
    cur_slices: current list (prefix) of slices in tensor that will be built
    sol_list (mutated): list to append to
    '''
    assert isinstance(row_ops,list)
    assert len(cur_slices)<=n_slices
    assert all(M.shape==(nrows,nclms) for M in cur_slices)
    
    if len(cur_slices)==n_slices:
        sol_list.append(np.array(cur_slices,dtype=int))
        return
    
    print('='*32+'\n'+f'depth: {len(cur_slices)}')
    good_pairs=[]
    for P in tqdm(row_ops):
        # find all inv'le nclms x nclms Q s.t. all(M==P@M@Q for M in cur_slices)
        sys_left=np.concatenate([
            (P@M)%MOD
            for M in cur_slices
        ],axis=0)
        sys_right=np.concatenate(cur_slices,axis=0)
        good_pairs.extend(
            (P,Q)
            for Q in linsys(sys_left,sys_right,MOD)
            if mat_rank(Q,MOD)==nclms
        )
            
    assert len(good_pairs)>0  # should at least contain the identity
    print(f'# good pairs={len(good_pairs)}')
    
    seen=set()
    canonicals=[]
    for M in all_tensors((nrows,nclms),MOD):
        code=tensor2code(M)
        if code not in seen:
            canonicals.append(M)
            seen.update(
                tensor2code((P@M@Q)%MOD)
                for P,Q in good_pairs
            )
    print(f'# canonicals={len(canonicals)}')
    
    for M in canonicals:
        dfs_canonicals_dim3(
            n_slices,nrows,nclms,MOD,row_ops,
            cur_slices+[M,],sol_list
        )

In [9]:
canonical_tensors=[]
A=np.zeros((3,3),dtype=int)
A[range(3),range(3)]=1
init_slices=[
    A,
]
display(init_slices)
dfs_canonicals_dim3(
    n_slices=3,nrows=3,nclms=3,MOD=2,
    row_ops=list(all_invle_mats(3,2)),
    cur_slices=init_slices,
    sol_list=canonical_tensors
)

[array([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])]

depth: 1


100%|████████████████████████████████████████| 168/168 [00:00<00:00, 958.12it/s]


# good pairs=168
# canonicals=14
depth: 2


100%|████████████████████████████████████████| 168/168 [00:00<00:00, 808.61it/s]


...

# good pairs=7
# canonicals=80
depth: 2


100%|████████████████████████████████████████| 168/168 [00:00<00:00, 936.77it/s]


# good pairs=7
# canonicals=80
depth: 2


100%|████████████████████████████████████████| 168/168 [00:00<00:00, 804.19it/s]

# good pairs=168
# canonicals=14


In [10]:
len(canonical_tensors)

1744

In [17]:
for T_target in tqdm(canonical_tensors):
    MOD=2
    rank_thresh=4
    
    if all(
        k_th_rref_prune0(rotate(T_target,sh),rank_thresh,MOD,k=2)!=NO_CPD_EXISTS
        for sh in range(len(T_target.shape))
    ):
        ret=cpd_search(
            T_target,rank_thresh,MOD,
            pruners={
                'rref':prune_rref,
                'ranksum':prune_contract_ranksum,
            },
            verbose=False
        )
        if ret is None:
            print(T_target)
            break

 14%|█████▋                                  | 246/1744 [01:41<10:16,  2.43it/s]

[[[1 0 0]
  [0 1 0]
  [0 0 1]]

 [[0 0 0]
  [0 0 1]
  [0 1 0]]

 [[0 0 1]
  [0 0 0]
  [1 1 0]]]


In [44]:
T_target=np.array(
[[[1,0,0],
  [0,1,0],
  [0,0,1],],

 [[0,0,0],
  [0,0,1],
  [0,1,0],],

 [[0,0,1],
  [0,0,0],
  [1,1,0],],]
,dtype=int)
display(T_target)

for ax in range(len(T_target.shape)):
    print(f'{ax=}')
    k_th_rref_prune0(rotate(T_target,ax),rank_thresh,MOD,k=2,verbose=True)    

array([[[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]],

       [[0, 0, 0],
        [0, 0, 1],
        [0, 1, 0]],

       [[0, 0, 1],
        [0, 0, 0],
        [1, 1, 0]]])

ax=0
[(array([0, 0, 1]), array([1, 1, 0])), (array([0, 0, 1]), array([1, 1, 1])), (array([0, 1, 0]), array([1, 0, 1])), (array([0, 1, 0]), array([1, 1, 1])), (array([0, 1, 1]), array([1, 0, 1])), (array([0, 1, 1]), array([1, 1, 0])), (array([1, 0, 1]), array([1, 1, 0])), (array([1, 0, 1]), array([1, 1, 1])), (array([1, 1, 0]), array([1, 1, 1]))]
[(array([0, 0, 1]), array([1, 1, 0])), (array([0, 1, 0]), array([1, 0, 1])), (array([0, 1, 1]), array([1, 0, 1]))]
ax=1
[(array([0, 1, 0]), array([1, 0, 1])), (array([0, 1, 0]), array([1, 1, 1])), (array([0, 1, 1]), array([1, 0, 0])), (array([0, 1, 1]), array([1, 0, 1])), (array([0, 1, 1]), array([1, 1, 0])), (array([0, 1, 1]), array([1, 1, 1])), (array([1, 0, 0]), array([1, 1, 1])), (array([1, 0, 1]), array([1, 1, 0])), (array([1, 0, 1]), array([1, 1, 1]))]
[(array([0, 1, 0]), array([1, 0, 1])), (array([0, 1, 1]), array([1, 0, 0])), (array([0, 1, 1]), array([1, 0, 1]))]
ax=2
[(array([0, 1, 0]), array([1, 0, 1])), (array([0, 1, 0]), array([1, 1

ax=0: 2-planes are [(array([0, 0, 1]), array([1, 1, 0])), (array([0, 1, 0]), array([1, 0, 1])), (array([0, 1, 1]), array([1, 0, 1]))]

create the span of each tuple, then take pairwise intersections: we get
* 0th, 1st: [1,1,1]
    * because the intersection of span([0,0,1],[1,1,0]) and span([0,1,0],[1,0,1])
    is span([1,1,1])
* 0th, 2nd: [1,1,0]
* 1st, 2nd: [1,0,1]

repeat for other axes

ax=1:
* [(array([0, 1, 0]), array([1, 0, 1])), (array([0, 1, 1]), array([1, 0, 0])), (array([0, 1, 1]), array([1, 0, 1]))]
* [1,1,1], [1,0,1], [0,1,1]

ax=2:
* [(array([0, 1, 0]), array([1, 0, 1])), (array([0, 1, 1]), array([1, 0, 0])), (array([0, 1, 1]), array([1, 0, 1]))] (same as ax=1)
* [1,1,1], [1,0,1], [0,1,1]

changing basis of T along each axis forms a new tensor
such that the k-planes that pass k-th order rref-pruning
are precisely the k-subsets of slices

(i.e. the k-planes form the standard basis)

In [48]:
T_target=np.array(
[[[1,0,0],
  [0,1,0],
  [0,0,1],],

 [[0,0,0],
  [0,0,1],
  [0,1,0],],

 [[0,0,1],
  [0,0,0],
  [1,1,0],],]
,dtype=int)

T_target_canonicalized=axis_ops(
    [
        np.array([[1,1,1],[1,1,0],[1,0,1]]),
        np.array([[1,1,1],[1,0,1],[0,1,1]]),
        np.array([[1,1,1],[1,0,1],[0,1,1]]),
    ],
    T_target,
    MOD=2
)

for sh in range(len(T_target_canonicalized.shape)):
    display(rotate(T_target_canonicalized,sh))

assert all(
    k_th_rref_prune0(rotate(T_target_canonicalized,sh),R=4,MOD=2,k=2)!=NO_CPD_EXISTS
    for sh in range(len(T_target_canonicalized.shape))
)
assert cpd_search(T_target_canonicalized,R=4,MOD=2) is None

array([[[0, 1, 0],
        [0, 0, 0],
        [0, 1, 1]],

       [[1, 1, 0],
        [1, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [1, 0, 1],
        [0, 0, 1]]])

array([[[0, 1, 0],
        [1, 1, 0],
        [0, 0, 0]],

       [[0, 1, 1],
        [0, 0, 0],
        [0, 0, 1]],

       [[0, 0, 0],
        [1, 0, 0],
        [1, 0, 1]]])

array([[[0, 0, 0],
        [1, 1, 0],
        [0, 1, 0]],

       [[1, 0, 1],
        [1, 0, 0],
        [0, 0, 0]],

       [[0, 0, 1],
        [0, 0, 0],
        [0, 1, 1]]])

t=2.0972641409607604
{'work': [344, 1]}


we notice that the axis-0 slices (i,:,:) seem to be antidiagonally shifted copies of the same matrix

so we do some extra coordinate permutations to make this symmetry more clear
(and convert to diagonal-shift symmetry, because that is a bit easier to notate)

In [57]:
A=np.array([
    [1,1,0],
    [0,1,0],
    [0,0,0]
],dtype=int)
shift_mat=np.roll(np.eye(3,dtype=int),shift=1,axis=0)
# A,shift_mat@A@shift_mat.T

sh_A=A
slices=[]
for p in range(3):
    slices.append(sh_A)
    sh_A=shift_mat@sh_A@shift_mat.T

T=np.array(slices)
for sh in range(len(T.shape)):
    display(sh,rotate(T,sh))

assert all(
    k_th_rref_prune0(rotate(T,sh),R=4,MOD=2,k=2)!=NO_CPD_EXISTS
    for sh in range(len(T.shape))
)
assert cpd_search(T,R=4,MOD=2) is None

0

array([[[1, 1, 0],
        [0, 1, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 1, 1],
        [0, 0, 1]],

       [[1, 0, 0],
        [0, 0, 0],
        [1, 0, 1]]])

1

array([[[1, 0, 1],
        [1, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [1, 1, 0],
        [0, 1, 0]],

       [[0, 0, 1],
        [0, 0, 0],
        [0, 1, 1]]])

2

array([[[1, 0, 0],
        [0, 0, 0],
        [1, 0, 1]],

       [[1, 1, 0],
        [0, 1, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 1, 1],
        [0, 0, 1]]])

t=2.0958749330602586
{'work': [344, 1]}


tensor also passes Laskowski pruning for R=4

In [59]:
def prune_lask0(T,R,MOD,verbose=False):
    assert len(T.shape)==3
    n0=T.shape[0]
    assert n0>0
    tot_rank=sum(
        mat_rank(axis_op(v.reshape((1,n0)),T,0,MOD)[0],MOD)
        for v in all_vecs(n0,MOD)
    )
    if verbose:
        ranks=np.zeros((MOD,)*n0,dtype=int)
        for v in all_vecs(n0,MOD):
            ranks[tuple(v)]=mat_rank(axis_op(v.reshape((1,n0)),T,0,MOD)[0],MOD)
        print(ranks)
    return (
        CANNOT_DETERMINE_IF_CPD_EXISTS
        if tot_rank <= R * (MOD**n0 - MOD**(n0-1))
        else NO_CPD_EXISTS
    )
prune_lask=expand_pruner_rotate(prune_lask0)

all(
    prune_lask(rotate(T,sh),R=4,MOD=2)!=NO_CPD_EXISTS
    for sh in range(len(T.shape))
)

True